Task 4: multi-object tracking challenge. The task aims to recover the trajectories of objects in each video frame.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio.v2 as io
import cv2
import torch
import os
import sys
from utils.utils_funcitons import *
device= "cuda" if torch.cuda.is_available() else "cpu"
from ultralytics import YOLO

In [2]:
dataset_path = r'.\dataset\Task4_Multi_Object_Tracking'
sequences_folder = os.path.join(dataset_path, 'sequences')
annotations_folder = os.path.join(dataset_path, 'annotations')
model_path = r'.\models\best_yolov8n.pt'

### YOLOv8 Multi-Object Tracker

In [11]:
model = YOLO(model_path)
objects = ['all']
number_class_list = []

if objects!=None:
    if objects!=['all']:
        for object in objects: 
            number_class_list.append(classes.index(object))
    elif objects == ['all']:
        number_class_list = list(range(len(classes)))

# Images folder path
img_folder = os.path.join(sequences_folder, 'uav0000117_02622_v')
img_names = sorted(os.listdir(img_folder))

# Video size
width, height = 1280, 720

# Codec and VideoWriter
codec = cv2.VideoWriter_fourcc(*'XVID')
fps = 20
output_video = cv2.VideoWriter(r'.\outputs\Task4\output_yolo.mp4', codec, fps, (width, height))

# Frame processing
for i, img in enumerate(img_names):
    img_fullpath = os.path.join(img_folder, img)
    frame = cv2.imread(img_fullpath)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    if frame is not None:
        # Redimensionar la imagen si es necesario
        if width and height:
            results = model.track(frame, persist=True, verbose=False, conf=0.3, iou=0.5, classes=number_class_list, imgsz=704, tracker=r'.\botsort_custom.yaml')
            annotated_frame = results[0].plot()               
            annotated_frame = cv2.resize(annotated_frame, (width, height))
            #output_video.write(cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))
            output_video.write(annotated_frame)

# Liberar recursos
output_video.release()
cv2.destroyAllWindows()